---  
title: "Papermill pipeline for MBSA-QIP analysis"  
subtitle: "An series of automated analyses with varying parameters"  
date: last-modified  
date-format: full  
author:  
  - name:  
      given: Pranav Kumar  
      family: Mishra  
    affiliations:  
      - ref: rushsurg  
      - ref: rushortho  
    corresponding: true  
    url: https://drpranavmishra.com  
    email: pranav_k_mishra@rush.edu  
    orcid: 0000-0001-5219-6269  
    role: "Post Doctoral Research Fellow"  
format:  
  html:  
    code-fold: true  

  pdf:
    documentclass: scrartcl
    toc: true
    code-fold: true
    papersize: letter
    toc-depth: 2
    margin-left: 30mm
    margin-right: 30mm
    highlight-style: github
    colorlinks: true
    default-image-extension: png 
execute:   
  enabled: false  
  echo: false  
  output: asis #true  
---

In [ ]:
# Papermill parameters
run_num = "001"

run_dict_param = {
    "001": {"CPT": {"codes": ["43644"]}, 
            "year_start": 2021, 
            "year_stop": 2022},
    "002": {"CPT": {"codes": ["43644", "43755"]}, 
            "year_start": 2021, 
            "year_stop": 2022},
                        
    "003": {"CPT": {"codes": ["43644", "43755"]}, 
            "year_start": 2018, 
            "year_stop": 2022},
}


save_notebooks = True
save_figures = True
save_tables = True

# General
verbosity = True
repository_root = "/home/pranav/work/pranavmishra90/courses/by-mishra/research-data-management"

In [ ]:
run_num = str(run_num)

In [ ]:
import papermill as pm
import os, socket

# Display
from IPython.display import display, Markdown, Latex


In [ ]:
# Get the computer name and the repository root

# Get the name of the computer, then save it to an environment variable
os.environ['computer_name']= socket.gethostname()

if len(os.environ['computer_name']) == 12:
    if os.path.exists('/home/coder/work'):
        
        is_docker = 'True'
        os.environ['is_docker']=is_docker
        os.chdir('/home/coder/work')
        os.environ['repo_root']=os.getcwd()
        print('This is a docker environment')

    
   
elif os.path.exists(repository_root) == True:
    is_docker = 'False'
    os.environ['is_docker'] = is_docker
    os.environ['repo_root'] = repository_root

else:
    is_docker = 'False'
    os.environ['is_docker'] = is_docker
    print(f"Error finding the repository root for computer {os.environ['computer_name']}")


print(f"The computer\'s name is {os.environ['computer_name']}")
os.chdir(os.environ['repo_root'])
print(f"The current directory has been set to {os.getcwd()}")

In [ ]:
def pm_paths(run_num, save_notebooks, run_dict_param):

    run_base_directory = 	os.path.join(os.environ['repo_root'], f"data/analysis/bariatric/runs")
    this_run_directory = os.path.join(run_base_directory, f"run_{run_num}")


    if save_notebooks == True:

        # Notebook directory
        notebook_base_path = os.path.join(
            os.environ["repo_root"], this_run_directory, "notebooks/"
        )
        if not os.path.exists(notebook_base_path):
            os.makedirs(notebook_base_path)
        else:
            #print(f"A notebook directory is already present: {notebook_base_path}")
            pass
        
        pm_output_nb = f"{notebook_base_path}Run{run_num}_analysis.ipynb"
    else:
        pm_output_nb = f"code/jupyter/papermill_tmp/Run{run_num}_analysis.ipynb"

    display(Markdown(f"""#### Run `{run_num}`:
    
- **The Jupyter Notebook**: {pm_output_nb}
- **Years**:  {run_dict_param[run_num]["year_start"]} - {run_dict_param[run_num]["year_stop"]}
- **CPT Codes**: {run_dict_param[run_num]["CPT"]}
"""))

    return pm_output_nb, run_base_directory, this_run_directory

In [ ]:
# Papermill loop

run_list = list(run_dict_param.keys())

for selected_run in run_list:
    pm_output_nb, run_base_directory, this_run_directory = pm_paths(selected_run, save_notebooks, run_dict_param)

    cpt_values = run_dict_param[selected_run]["CPT"]["codes"]



    pm.execute_notebook("code/jupyter/bariatric.ipynb",
                        pm_output_nb,
                        parameters=dict(
                            run_num=int(selected_run),
                            CPT_filter=run_dict_param[selected_run]["CPT"]["codes"],
                            year_start=run_dict_param[selected_run]["year_start"],
                            year_stop=run_dict_param[selected_run]["year_stop"],
                            save_figures=save_figures,
                            save_tables=save_tables,
                        )
    )